In [7]:
# Playlist fetcher for checking what playlists a song-link I want to add is within

import os
from pprint import pprint
import json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [8]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('SPOTIPY_CLIENT_ID_CONTAINED_PLAYLISTS_FETCHER'), client_secret=os.getenv('SPOTIPY_CLIENT_SECRET_CONTAINED_PLAYLISTS_FETCHER'))
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

library_account = os.getenv('SPOTIFY_PLUS_SECONDARY_ACCOUNT_USER_ID')

In [9]:
playlist_ids = []

immediate_to_sort_id = ''
library_to_sort_id = ''

genre_ids = []
archived_mixtape_ids = []
archived_record_ids = []

current_page = sp.user_playlists(library_account)
while current_page:
    for playlist in current_page['items']:
        id = playlist['id']
        playlist_ids.append(id)

        if playlist['name'].startswith('[G]'):
            genre_ids.append(id)
        elif playlist['name'].startswith('[AM]'):
            archived_mixtape_ids.append(id)
        elif playlist['name'].startswith('[AR]'):
            archived_record_ids.append(id)
        elif playlist['name'].startswith('[1]'):
            immediate_to_sort_id = id
        elif playlist['name'].startswith('[2]'):
            library_to_sort_id = id

    if current_page['next']:
        current_page = sp.next(current_page)
    else:
        current_page = None

In [10]:
def get_simplified_tracks(playlist_id):
    simplified_tracks = []

    current_page = sp.playlist_items(f'spotify:playlist:{playlist_id}', fields='items.track.id,items.track.is_local,items.track.name,next')
    while current_page:
        for item in current_page['items']:
            track = item['track']
            new_track = {}

            new_track['is_local'] = track['is_local']

            if not track['is_local']:
                new_track['id'] = track['id']
            else:
                new_track['id'] = None
            
            new_track['name'] = track['name']
            
            simplified_tracks.append(new_track)

        if current_page['next']:
            current_page = sp.next(current_page)
        else:
            current_page = None
    
    return simplified_tracks

In [11]:
immediate_to_sort_tracks = get_simplified_tracks(immediate_to_sort_id)
library_to_sort_tracks = get_simplified_tracks(library_to_sort_id)

genre_id_to_tracks = {genre_id: get_simplified_tracks(genre_id) for genre_id in genre_ids}
archived_mixtape_id_to_tracks = {am_id: get_simplified_tracks(am_id) for am_id in archived_mixtape_ids}
archived_record_id_to_tracks = {ar_id: get_simplified_tracks(ar_id) for ar_id in archived_record_ids}

In [15]:
with open("saved_libraries/immediate_to_sort_tracks.json", "w", encoding="utf-8") as outFile:
    json.dump(immediate_to_sort_tracks, outFile, ensure_ascii=False, indent=4)
with open("saved_libraries/library_to_sort_tracks.json", "w", encoding="utf-8") as outFile:
    json.dump(library_to_sort_tracks, outFile, ensure_ascii=False, indent=4)

with open("saved_libraries/genre_id_to_tracks.json", "w", encoding="utf-8") as outFile:
    json.dump(genre_id_to_tracks, outFile, ensure_ascii=False, indent=4)
with open("saved_libraries/archived_mixtape_id_to_tracks.json", "w", encoding="utf-8") as outFile:
    json.dump(archived_mixtape_id_to_tracks, outFile, ensure_ascii=False, indent=4)
with open("saved_libraries/archived_record_id_to_tracks.json", "w", encoding="utf-8") as outFile:
    json.dump(archived_record_id_to_tracks, outFile, ensure_ascii=False, indent=4)